### 管理静态文件
+ 项目中的CSS、图片、js都是静态文件
+ 配置静态文件
+ 在settings 文件中定义静态内容
+ STATIC_URL = '/static/'
+ STATICFILES_DIRS = [
+     os.path.join(BASE_DIR, 'static'),
+ ]
+ STATIC_URL 设置访问静态文件对应的url
+ STATICFILES_DIRS 设置静态文件所在的物理目录
+ 动态生成静态文件的路径<br><br>

+ 在项目根目录下创建static目录，再创建当前应用名称的目录
+ mysite/static/myapp/
+ 在模板中可以使用硬编码
+ /static/my_app/myexample.jpg
+ 在模板中可以使用static编码
+ { % load static from staticfiles %}
+ <img src="{ % static "my_app/myexample.jpg" %}" alt="My image"/>

### 中间件
+ 中间件函数是Django框架预留的函数接口，是一个轻量级、底层的插件系统，可以介入Django的请求和响应处理过程，修改Django的输入或输出
+ 激活：添加到Django配置文件中的MIDDLEWARE_CLASSES元组中
+ 每个中间件组件是一个独立的Python类，可以定义下面方法中的一个或多个<br><br>

![title](img/中间件执行流程.png)
+ _init _：无需任何参数，服务器响应第一个请求的时候调用一次，用于确定是否启用当前中间件<br><br>

+ process_request(request)：执行视图之前被调用，在每个请求上调用，返回None或HttpResponse对象。
+ 总结process_request
+ 1.中间件的process_request方法是在执行视图函数之前执行的。
+ 2.当配置多个中间件时，会按照MIDDLEWARE中的注册顺序，也就是列表的索引值，从前到后依次执行的。
+ 3.不同中间件之间传递的request都是同一个对象
+ 4.返回值 1.返回None,不错任何处理直接进行下一步
+ 5.返回值 2.返回响应对象，直接跳出（后续中间件的process_request、不执行urls.py和views.py）返回响应
+ 多个中间件中的process_response方法是按照MIDDLEWARE中的注册顺序倒序执行的，也就是说第一个中间件的process_request方法首先执行，而它的process_response方法最后执行，最后一个中间件的process_request方法最后一个执行，它的process_response方法是最先执行。<br><br>

+ process_view(request, view_func, view_args, view_kwargs)：调用视图之前被调用，在每个请求上调用，返回None或HttpResponse对象
+ request是HttpRequest对象。
+ view_func是Django即将使用的视图函数。 （它是实际的函数对象，而不是函数的名称作为字符串。）
+ view_args是将传递给视图的位置参数的列表.
+ view_kwargs是将传递给视图的关键字参数的字典。 view_args和view_kwargs都不包含第一个视图参数（request）。
+ Django会在调用视图函数之前调用process_view方法。<br><br>

+ 总结:process_view
+ 1.在views.py返回响应对象之后执行
+ 2.执行的顺序按照在列表中注册的倒序依次执行
+ 3.返回值必须要有返回值，返回要是响应对象<br><br>

+ process_template_response(request, response)：在视图刚好执行完毕之后被调用，在每个请求上调用，返回实现了render方法的响应对象

+ process_response(request, response)：所有响应返回浏览器之前被调用，在每个请求上调用，返回HttpResponse对象
+ 多个中间件中的process_response方法是按照MIDDLEWARE中的注册顺序倒序执行的，也就是说第一个中间件的process_request方法首先执行，而它的process_response方法最后执行，最后一个中间件的process_request方法最后一个执行，它的process_response方法是最先执行。<br><br>

+ process_exception(request,response,exception)：当视图抛出异常时调用，在每个请求上调用，返回一个HttpResponse对象

#### 中间件建立过程
+ 1.使用中间件(在booktest目录下新建middleware.py文件)
+ 2.定义中间件类<br><br>



#### 示例：自定义异常处理
+ 使用中间件，可以干扰整个处理过程，每次请求中都会执行中间件的这个方法
+ 与settings.py同级目录下创建myexception.py文件，定义类MyException，实现process_exception方法
+ from django.http import HttpResponse
+ class MyException():
+     def process_exception(request,response, exception):
+         return HttpResponse(exception.message)
+ 将类MyException注册到settings.py中间件中
+ MIDDLEWARE_CLASSES = (
+     'test1.myexception.MyException',
+     ...
+ )
+ 定义视图，并发生一个异常信息，则会运行自定义的异常处理

#### Django框架的执行过程:
+ 1.request请求经过WSGI后，先进入中间件，依然开始先走process_request函数，然后走路由关系映射后，这里注意并没有直接进入视图函数，而是从头开始执行process_view()函数；然后再去执行与urls.py匹配的视图函数；<br><br>
+ 2.如果视图函数没有报错，那就直接挨个反过来从最后一个中间件开始，依次将返回的实例对象(也就是我们在视图函数中写的 return HttpResponse()等等)传递给每个中间件的process_response函数；最后再交给客户端浏览器；<br><br>
+ 3.如果执行视图函数出错，那就反过来从最后一个中间件开始，将错误信息传递给每个中间件的process_exception()函数，走完所有后，然后最终再走procss_response后，最终再交给客户端浏览器注意：视图函数的错误是由process_exception()函数处理的，从最后一个中间件开始，依次逐级提交捕捉到的异常然后最终交给procss_response()函数，将最终的错误信息交给客户端浏览器。<br><br>